In [ ]:
from PIL import Image
import numpy as np
from numpy import asarray
import os
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.linalg import eigh
from sklearn.metrics import accuracy_score

In [ ]:
def getTrainTestSplits(folderName,trainSize):
    allFolders = []
    for folders in os.listdir(folderName):
        if folders[0] == 's':
            allFolders.append(folders)


    firstX = 1
    firstY = 1
    Y_train = []
    Y_test = []
    
    for folder in allFolders:
        loc = folderName+"/"+folder
        count = 0
        folnum = int(folder[1:])
        for file in os.listdir(loc):
            fileLocation = loc + "/" + file
            image = Image.open(fileLocation)
            pixels = asarray(image)
            pixels = np.reshape(pixels,[1,pixels.shape[0]*pixels.shape[1]])

            if(count < trainSize):
                if(firstX):
                    X_train = pixels
                    firstX = 0
                else:
                    X_train = np.vstack([X_train,pixels])
                Y_train.append(folnum)
            else:
                if(firstY):
                    X_test = pixels
                    firstY = 0
                else:
                    X_test = np.vstack([X_test,pixels])
                Y_test.append(folnum)


            count = count + 1

    return X_train,Y_train,X_test,Y_test

In [ ]:
X_train,Y_train,X_test,Y_test = getTrainTestSplits("./ATnT",6)

In [ ]:
def topPrincipalComponents(eigen_values,eigen_faces,num_of_components):
    ef = eigen_faces[:,:num_of_components]
    return ef
def principalComponentAnalysis(X):
    global all_means
    all_means = np.mean(X,axis=0)
    X = X - all_means
    coVar = np.cov(X.T)
    values,vectors = eigh(coVar)    
    sorted_index = (-values).argsort()
    sorted_eigen_faces = vectors[:,sorted_index]
    return values,sorted_eigen_faces
eigen_values,eigen_faces = principalComponentAnalysis(X_train)

In [ ]:
def imageReconstruction(row,all_means,ef,Y_train,img_num,cmp,j,tmp):
    row = row - all_means
    xk = np.matmul(row,ef)
    row = np.matmul(xk,ef.T)
    row = row + all_means
    row = np.reshape(row,[112,92])
    plt.subplot(6,6,tmp+1)
    plt.imshow(row)
    plt.subplots_adjust(right=1.2, top=1.2)
    plt.tick_params(labelleft='off', labelbottom='off', bottom='off',top='off',right='off',left='off', which='both')
    if(j == 5):
        print('Face Number: '+str(img_num))
        plt.show()
    img = Y_train[img_num]

In [ ]:
ef = topPrincipalComponents(eigen_values,eigen_faces,50)
tmp = 0
for i in range(0,ef.shape[1]):
    tf = ef[:,i]
    fig,ax=pyplot.subplots(1,1,figsize=(6,6))
    ax.imshow(tf.reshape((112,92)))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('Average Face')